In [22]:
using JuMP
using HiGHS
using Ipopt
using Optimization

In [23]:
model = Model()

A JuMP Model
├ solver: none
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [24]:
# supuestos 
# el problema tiene 6 filas/personas
# y también se tiene 7 columnas/tareas

# El problema es de tipo #tareas > #personas
# osea #columnas > #tareas

# eso quiere decir que es necesario agregar una "persona"
# extra (una fila de costo 0)

costos = [
    3  8  2 10  3  3  9;
    2  2  7  6  5  2  7;
    5  6  4  5  6  6  6;
    4  2  7  5  9  4  7;
   10  3  8  4  2  3  5;
    3  5  4  2  3  7  8;
    0  0  0  0  0  0  0
]

# variables 
@variable(model, x[1:7, 1:7] >= 0, Bin)

# constraints 
@constraint(model, sum(x[1,j] for j in 1:7) == 1)
@constraint(model, sum(x[2,j] for j in 1:7) == 1)
@constraint(model, sum(x[3,j] for j in 1:7) == 1)
@constraint(model, sum(x[4,j] for j in 1:7) == 1)
@constraint(model, sum(x[5,j] for j in 1:7) == 1)
@constraint(model, sum(x[6,j] for j in 1:7) == 1)
@constraint(model, sum(x[7,j] for j in 1:7) == 1)

@constraint(model, sum(x[i,1] for i in 1:7) == 1)
@constraint(model, sum(x[i,2] for i in 1:7) == 1)
@constraint(model, sum(x[i,3] for i in 1:7) == 1)
@constraint(model, sum(x[i,4] for i in 1:7) == 1)
@constraint(model, sum(x[i,5] for i in 1:7) == 1)
@constraint(model, sum(x[i,6] for i in 1:7) == 1)
@constraint(model, sum(x[i,7] for i in 1:7) == 1)


# objective function
@objective(model, Min, sum(costos[i,j] * x[i,j] for i in 1:7, j in 1:7))

3 x[1,1] + 8 x[1,2] + 2 x[1,3] + 10 x[1,4] + 3 x[1,5] + 3 x[1,6] + 9 x[1,7] + 2 x[2,1] + 2 x[2,2] + 7 x[2,3] + 6 x[2,4] + 5 x[2,5] + 2 x[2,6] + 7 x[2,7] + 5 x[3,1] + 6 x[3,2] + 4 x[3,3] + 5 x[3,4] + 6 x[3,5] + 6 x[3,6] + 6 x[3,7] + 4 x[4,1] + 2 x[4,2] + 7 x[4,3] + 5 x[4,4] + 9 x[4,5] + 4 x[4,6] + 7 x[4,7] + 10 x[5,1] + 3 x[5,2] + 8 x[5,3] + 4 x[5,4] + 2 x[5,5] + 3 x[5,6] + 5 x[5,7] + 3 x[6,1] + 5 x[6,2] + 4 x[6,3] + 2 x[6,4] + 3 x[6,5] + 7 x[6,6] + 8 x[6,7]

In [25]:
set_optimizer(model, HiGHS.Optimizer)
optimize!(model)

Running HiGHS 1.11.0 (git hash: 364c83a51e): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 14 rows; 49 cols; 98 nonzeros; 49 integer variables (49 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+00, 1e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
14 rows, 49 cols, 98 nonzeros  0s
14 rows, 49 cols, 98 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   14 rows
   49 cols (49 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   98 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constrai

In [26]:
# print solution
print("x = ", value.(x), "\n")
for i in 1:7 

    for j in 1:7
        print(value(x[i, j]), "   ")
        
    end
    println()
end

print("objective value = ", objective_value(model))

# final assignation
println("")
println("\nAsignaciones")
for i in 1:7  
    for j in 1:7
        if value(x[i, j]) == 1.0
            println("Persona $i → Tarea $j (Costo: $(costos[i, j]))")
        end
    end
end

println("\n=== Costo minimo ===")
println(objective_value(model))
println("Ya que la persona 7 es la persona que tiene la tarea 7, quiere decir que esa es la atarea que no se va a realizar")

x = [0.0 0.0 1.0 0.0 -0.0 -0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 1.0 0.0 -0.0 -0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0; -0.0 -0.0 0.0 0.0 0.0 0.0 1.0]
0.0   0.0   1.0   0.0   -0.0   -0.0   0.0   
0.0   0.0   0.0   0.0   0.0   1.0   0.0   
1.0   0.0   -0.0   -0.0   0.0   0.0   0.0   
0.0   1.0   0.0   0.0   0.0   0.0   0.0   
0.0   0.0   0.0   0.0   1.0   0.0   0.0   
0.0   0.0   0.0   1.0   0.0   0.0   0.0   
-0.0   -0.0   0.0   0.0   0.0   0.0   1.0   
objective value = 15.0

Asignaciones
Persona 1 → Tarea 3 (Costo: 2)
Persona 2 → Tarea 6 (Costo: 2)
Persona 3 → Tarea 1 (Costo: 5)
Persona 4 → Tarea 2 (Costo: 2)
Persona 5 → Tarea 5 (Costo: 2)
Persona 6 → Tarea 4 (Costo: 2)
Persona 7 → Tarea 7 (Costo: 0)

=== Costo minimo ===
15.0
Ya que la persona 7 es la persona que tiene la tarea 7, quiere decir que esa es la atarea que no se va a realizar
